# xdbg - Tutorial
[Home](index.html)

This tutorial is currently presented as a Jupyter notebook, but the interaction model works best with a Jupyter-enabled text editor like Atom.

To begin, you need to import xdbg into your IPython session (either in the console, or running inside a Jupyter environment)

In [1]:
import xdbg
from xdbg import debugger

When loaded, IPython is extended with a few extra magics, including `%break` and `%scope`.

## Breakpoints inside functions

The `%break` magic lets you pause the execution of a Python program inside a function, and execute interactive statements at that point.

For example:

In [2]:
def foo(n):
    %break

In [7]:
val = foo(5)

[xdbg] Entered: <__main__>.foo


The message above indicates that the scope of the interactive iterpreter has changed.

All commands now target the scope inside the function. For example, the variable `n` is now defined.

In [4]:
n

5

Any variables defined now are also defined inside the local scope. For example:

In [5]:
x = 5
'x' in globals()

False

Issuing a `return` statement exits the function scope

In [6]:
return 5

[xdbg] Exited: <__main__>.foo


In [8]:
val

5

In [9]:
x

NameError: name 'x' is not defined

As you see, execution continued (`val` is set), and the local variable `x` is no longer in scope.

## Working with modules

`xdbg` alse lets you move scope inside any module. Currently, this is only accessible via filename (to provide integration to text editors like Atom).

We are currently in the main scope, and the variable `xdbg.x` is not defined

In [10]:
__name__

'__main__'

In [11]:
xdbg.x

AttributeError: module 'xdbg' has no attribute 'x'

Now we can switch into the `__init__.py` file of `xdbg` and define `x`.

In [12]:
%scope ~/dev/xdbg/doc/xdbg/__init__.py

In [13]:
__name__

'xdbg'

In [14]:
x = 5

In [15]:
%scope # No arguments or invalid arguments returns to main

In [16]:
__name__

'__main__'

In [17]:
xdbg.x

5

`xdbg` also lets you define path aliases. These were designed to resolve situations with symlinks or when using remote filesystems.

For example:

In [18]:
debugger.path_mapping['~/dev/xdbg'] = '~/dev/xdbg/doc/xdbg'

In [19]:
%scope ~/dev/xdbg/__init__.py

In [20]:
__name__

'xdbg'

In [21]:
%scope
__name__

'__main__'

## Atom integration

The [deuterium package](https://atom.io/packages/deuterium) for the [atom text editor](http://atom.io/) integrates with `xdbg`.

It automatically switches scope to whatever file is in focus in the editor, and also shows the current scope in the status bar. Additional features are currently in development (let me know if there is anything you need!)